## Connecting India_GTSG Database

In [1]:
import pyodbc

In [2]:
DB = {'servername': 'AW02PSQLC007',
    'database': 'India_GTSG'}
conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

In [3]:
mycursor = conn.cursor()

## RFT_QUERY

In [4]:
## Adding rft excel data will be done in sql sever only

In [5]:
rft_query = ["""SELECT * INTO RFT_P6 FROM Sheet1$""",
             """DELETE from rft_p6 where "Resource (Utilization Target %)/Job Code" like '%Inactive%';""",
             """ALTER TABLE rft_p6 ADD Resource_Name VARCHAR(255);""",
             """ALTER TABLE rft_p6 ADD Employee_code VARCHAR(255);""",
             """ALTER TABLE rft_p6 ADD "Project Number" VARCHAR(255);""",
             """ALTER TABLE rft_p6 ADD Utilization_Target INT;""", 
             """ALTER TABLE rft_p6 ADD Utilization_Target_percent VARCHAR(255);""",
             """UPDATE rft_p6 set "Project Number"=CASE WHEN "Project" LIKE '%-%' then  RIGHT("Project", CHARINDEX(' ', (reverse("Project")))-1) ELSE 'NA' END""",
             """UPDATE rft_p6 SET Resource_Name=CASE
                WHEN "Resource (Utilization Target %)/Job Code" LIKE '%-%' 
                THEN LEFT("Resource (Utilization Target %)/Job Code", 
                CHARINDEX(RIGHT("Resource (Utilization Target %)/Job Code", 
                CHARINDEX('-', (reverse("Resource (Utilization Target %)/Job Code")))+1),
                (("Resource (Utilization Target %)/Job Code")))) ELSE "Resource (Utilization Target %)/Job Code"
                END """,
            """UPDATE rft_p6 set Utilization_Target_percent=CASE WHEN "Resource (Utilization Target %)/Job Code" LIKE '%-%' then  RIGHT("Resource (Utilization Target %)/Job Code", 
               CHARINDEX('(', (reverse("Resource (Utilization Target %)/Job Code")))) ELSE 'NA' END""",
            """UPDATE rft_p6 SET Utilization_Target=CASE
                WHEN "Resource (Utilization Target %)/Job Code" LIKE '%-%' 
                THEN dbo.UDF_ExtractNumbers(Utilization_Target_percent) 
                ELSE 0 END""",
            """UPDATE rft_p6 SET employee_code=CASE
                WHEN "Resource (Utilization Target %)/Job Code" LIKE '%-%' 
                THEN   
                SUBSTRING("Resource (Utilization Target %)/Job Code",
                LEN(LEFT("Resource (Utilization Target %)/Job Code",
                CHARINDEX('-', "Resource (Utilization Target %)/Job Code")+1)),
                LEN("Resource (Utilization Target %)/Job Code") - LEN(LEFT("Resource (Utilization Target %)/Job Code",
                CHARINDEX('-', "Resource (Utilization Target %)/Job Code"))) - LEN(RIGHT("Resource (Utilization Target %)/Job Code",
                CHARINDEX(' ', (REVERSE("Resource (Utilization Target %)/Job Code"))))))
                ELSE "Resource (Utilization Target %)/Job Code"
                END""",
            """UPDATE rft_p6 SET employee_code=CASE
                WHEN "Resource (Utilization Target %)/Job Code" LIKE '%-%' 
                THEN dbo.UDF_ExtractNumbers(employee_code) 
                ELSE 0 END""",
            """INSERT INTO UT ("Resource (UTilization Target %)/Job Code")
                 SELECT distinct T1."Resource (UTilization Target %)/Job Code" 
                 FROM rft_p6 T1
                 LEFT JOIN UT t2
                 ON T1."Resource (UTilization Target %)/Job Code" = T2."Resource (UTilization Target %)/Job Code" WHERE T2."Resource (UTilization Target %)/Job Code" IS NULL;""",
             """UPDATE UT SET Resource_Name=CASE
                 WHEN "Resource (UTilization Target %)/Job Code" LIKE '%-%' 
                 THEN left("Resource (UTilization Target %)/Job Code", 
                 CHARINDEX(right("Resource (UTilization Target %)/Job Code", 
                 CHARINDEX('-', (reverse("Resource (UTilization Target %)/Job Code")))+1),
                 (("Resource (UTilization Target %)/Job Code")))) ELSE "Resource (UTilization Target %)/Job Code"
                 END;""",
             """update UT SET UTilization_Target_percent=case when "Resource (UTilization Target %)/Job Code" LIKE '%-%' then  right("Resource (UTilization Target %)/Job Code", 
                 CHARINDEX('(', (reverse("Resource (UTilization Target %)/Job Code")))) ELSE 'NA' end;""",
                 """UPDATE UT SET UTilization_Target=CASE
                 WHEN "Resource (UTilization Target %)/Job Code" LIKE '%-%' 
                 THEN dbo.UDF_ExtractNumbers(UTilization_Target_percent) 
                 ELSE 0 end;""",
             """UPDATE UT SET employee_code=CASE
                 WHEN "Resource (UTilization Target %)/Job Code" LIKE '%-%' 
                 THEN   
                 SUBSTRING("Resource (UTilization Target %)/Job Code",
                 LEN(LEFT("Resource (UTilization Target %)/Job Code",
                 CHARINDEX('-', "Resource (UTilization Target %)/Job Code")+1)),
                 LEN("Resource (UTilization Target %)/Job Code") - LEN(LEFT("Resource (UTilization Target %)/Job Code",
                 CHARINDEX('-', "Resource (UTilization Target %)/Job Code"))) - LEN(RIGHT("Resource (UTilization Target %)/Job Code",
                 CHARINDEX(' ', (REVERSE("Resource (UTilization Target %)/Job Code"))))))
                 ELSE "Resource (UTilization Target %)/Job Code"
                 END;""",
             """UPDATE UT SET employee_code=CASE
                 WHEN "Resource (UTilization Target %)/Job Code" LIKE '%-%' 
                 THEN dbo.UDF_ExtractNumbers(employee_code) 
                 ELSE 0 end;""",
             """delete from UT where "UTilization_Target_percent" LIKE '%Inactive%';""",
             """WITH CTE AS  
                (  
                   SELECT *, ROW_NUMBER() over (PARTITION BY "Resource_Name" ORDER BY "Resource_Name") as rk  
                   FROM UT
                )  
                DELETE FROM CTE WHERE rk >1""",
            """UPDATE ut
SET "Jan-23"= (40*4*Utilization_Target)/100.0, "Feb-23"= (40*4*Utilization_Target)/100.0,
    "Mar-23"= (40*5*Utilization_Target)/100.0, "Apr-23"= (40*4*Utilization_Target)/100.0,
    "May-23"= (40*4*Utilization_Target)/100.0, "Jun-23"= (40*5*Utilization_Target)/100.0,
    "Jul-23"= (40*4*Utilization_Target)/100.0, "Aug-23"= (40*4*Utilization_Target)/100.0,
    "Sep-23"= (40*5*Utilization_Target)/100.0, "Oct-23"= (40*4*Utilization_Target)/100.0,
    "Nov-23"= (40*4*Utilization_Target)/100.0, "Dec-23"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-24"= (40*4*Utilization_Target)/100.0, "Feb-24"= (40*4*Utilization_Target)/100.0,
    "Mar-24"= (40*5*Utilization_Target)/100.0, "Apr-24"= (40*4*Utilization_Target)/100.0,
    "May-24"= (40*4*Utilization_Target)/100.0, "Jun-24"= (40*5*Utilization_Target)/100.0,
    "Jul-24"= (40*4*Utilization_Target)/100.0, "Aug-24"= (40*4*Utilization_Target)/100.0,
    "Sep-24"= (40*5*Utilization_Target)/100.0, "Oct-24"= (40*4*Utilization_Target)/100.0,
    "Nov-24"= (40*4*Utilization_Target)/100.0, "Dec-24"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-25"= (40*4*Utilization_Target)/100.0, "Feb-25"= (40*4*Utilization_Target)/100.0,
    "Mar-25"= (40*5*Utilization_Target)/100.0, "Apr-25"= (40*4*Utilization_Target)/100.0,
    "May-25"= (40*4*Utilization_Target)/100.0, "Jun-25"= (40*5*Utilization_Target)/100.0,
    "Jul-25"= (40*4*Utilization_Target)/100.0, "Aug-25"= (40*4*Utilization_Target)/100.0,
    "Sep-25"= (40*5*Utilization_Target)/100.0, "Oct-25"= (40*4*Utilization_Target)/100.0,
    "Nov-25"= (40*4*Utilization_Target)/100.0, "Dec-25"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-26"= (40*4*Utilization_Target)/100.0, "Feb-26"= (40*4*Utilization_Target)/100.0,
    "Mar-26"= (40*5*Utilization_Target)/100.0, "Apr-26"= (40*4*Utilization_Target)/100.0,
    "May-26"= (40*4*Utilization_Target)/100.0, "Jun-26"= (40*5*Utilization_Target)/100.0,
    "Jul-26"= (40*4*Utilization_Target)/100.0, "Aug-26"= (40*4*Utilization_Target)/100.0,
    "Sep-26"= (40*5*Utilization_Target)/100.0, "Oct-26"= (40*4*Utilization_Target)/100.0,
    "Nov-26"= (40*4*Utilization_Target)/100.0, "Dec-26"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-27"= (40*4*Utilization_Target)/100.0, "Feb-27"= (40*4*Utilization_Target)/100.0,
    "Mar-27"= (40*5*Utilization_Target)/100.0, "Apr-27"= (40*4*Utilization_Target)/100.0,
    "May-27"= (40*4*Utilization_Target)/100.0, "Jun-27"= (40*5*Utilization_Target)/100.0,
    "Jul-27"= (40*4*Utilization_Target)/100.0, "Aug-27"= (40*4*Utilization_Target)/100.0,
    "Sep-27"= (40*5*Utilization_Target)/100.0, "Oct-27"= (40*4*Utilization_Target)/100.0,
    "Nov-27"= (40*4*Utilization_Target)/100.0, "Dec-27"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-28"= (40*4*Utilization_Target)/100.0, "Feb-28"= (40*4*Utilization_Target)/100.0,
    "Mar-28"= (40*5*Utilization_Target)/100.0, "Apr-28"= (40*4*Utilization_Target)/100.0,
    "May-28"= (40*4*Utilization_Target)/100.0, "Jun-28"= (40*5*Utilization_Target)/100.0,
    "Jul-28"= (40*4*Utilization_Target)/100.0, "Aug-28"= (40*4*Utilization_Target)/100.0,
    "Sep-28"= (40*5*Utilization_Target)/100.0, "Oct-28"= (40*4*Utilization_Target)/100.0,
    "Nov-28"= (40*4*Utilization_Target)/100.0, "Dec-28"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-29"= (40*4*Utilization_Target)/100.0, "Feb-29"= (40*4*Utilization_Target)/100.0,
    "Mar-29"= (40*5*Utilization_Target)/100.0, "Apr-29"= (40*4*Utilization_Target)/100.0,
    "May-29"= (40*4*Utilization_Target)/100.0, "Jun-29"= (40*5*Utilization_Target)/100.0,
    "Jul-29"= (40*4*Utilization_Target)/100.0, "Aug-29"= (40*4*Utilization_Target)/100.0,
    "Sep-29"= (40*5*Utilization_Target)/100.0, "Oct-29"= (40*4*Utilization_Target)/100.0,
    "Nov-29"= (40*4*Utilization_Target)/100.0, "Dec-29"= (40*5*Utilization_Target)/100.0""",
          """UPDATE ut
SET "Jan-30"= (40*4*Utilization_Target)/100.0, "Feb-30"= (40*4*Utilization_Target)/100.0,
    "Mar-30"= (40*5*Utilization_Target)/100.0, "Apr-30"= (40*4*Utilization_Target)/100.0,
    "May-30"= (40*4*Utilization_Target)/100.0, "Jun-30"= (40*5*Utilization_Target)/100.0,
    "Jul-30"= (40*4*Utilization_Target)/100.0, "Aug-30"= (40*4*Utilization_Target)/100.0,
    "Sep-30"= (40*5*Utilization_Target)/100.0, "Oct-30"= (40*4*Utilization_Target)/100.0,
    "Nov-30"= (40*4*Utilization_Target)/100.0, "Dec-30"= (40*5*Utilization_Target)/100.0"""]

In [6]:
for i in rft_query:
    mycursor.execute(i)

In [7]:
conn.commit()

In [8]:
import pandas as pd
list1 =[]
data = pd.DataFrame(list1, columns=['Executed'])
data.to_excel('Executed.xlsx')